In [1]:
import pymongo 
import fitz  # PyMuPDF
import re
from sklearn.feature_extraction.text import TfidfVectorizer # import sklearn
# import nltk

In [2]:
import os
import re
# from nltk.corpus import stopwords
import nltk

# nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from pdfminer.high_level import extract_text
from collections import defaultdict
import json
import joblib

In [3]:
# !pip install pdfminer.six
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
client = pymongo.MongoClient('mongodb+srv://triet:1@cluster0.cjzmm.mongodb.net/')
db = client.sample_mflix
collection = db.news
print(collection)

Collection(Database(MongoClient(host=['cluster0-shard-00-00.cjzmm.mongodb.net:27017', 'cluster0-shard-00-02.cjzmm.mongodb.net:27017', 'cluster0-shard-00-01.cjzmm.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', replicaset='atlas-ogfiv4-shard-0', tls=True), 'sample_mflix'), 'news')


In [5]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
vectorizer = TfidfVectorizer(max_features=5000)
vectorizer = joblib.load('vectorizer.joblib')

In [11]:
file_path = 'inverted_index_output_sorted.json'

with open(file_path, 'r', encoding='utf-8') as f:
    inverted_index = json.load(f)

In [12]:
def preprocess_query(query):
    """Preprocess the query to match the terms used in the inverted index."""
    # Lowercase and remove special characters
    query = query.lower()
    query = re.sub(r'\W+', ' ', query)
    return query.split()

def extract_keywords_from_query(query, vectorizer, inverted_index, num_results=10):
    """
    Extract keywords from a query, search the inverted index, and return relevant documents.
    
    Parameters:
        query (str): The user query.
        vectorizer (TfidfVectorizer): The vectorizer used to build the inverted index.
        inverted_index (dict): The inverted index containing keywords, document IDs, and scores.
        num_results (int): Number of top results to return.
    Returns:
        List of relevant documents with scores.
    """
    # Preprocess the query
    processed_query = preprocess_query(query)
    
    # Vectorize the query to identify relevant keywords
    query_vector = vectorizer.transform([' '.join(processed_query)]).toarray()[0]
    feature_array = vectorizer.get_feature_names_out()
    
    # Extract keywords with non-zero TF-IDF scores
    query_keywords = {feature_array[i]: query_vector[i] for i in range(len(feature_array)) if query_vector[i] > 0}
    
    # Search in the inverted index
    results = defaultdict(float)
    for keyword, score in query_keywords.items():
        if keyword in inverted_index:
            for doc_entry in inverted_index[keyword]:
                doc_id = str(doc_entry['doc_id'])
                results[doc_id] += score * doc_entry['score']  # Combine query and document scores

    # Sort results by relevance
    sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)[:num_results]
    
    return sorted_results

In [8]:
# import pickle
# with open('vectorizer.pkl', 'rb') as vectorizer:
#     pickle.load(vectorizer)  

In [13]:
query = "artificial intelligence and deep learning"

# with open('vectorizer.pkl', 'rb') as f:
#     pickle.load(vectorizer, f)  
    
results = extract_keywords_from_query(query, vectorizer, inverted_index, num_results=5)

# Display results
print("Query Results:")
for doc_id, score in results:
    print(f"Document ID: {doc_id}, Relevance Score: {score:.4f}")

Query Results:
Document ID: 672c4842d2cd8acad1c138e9, Relevance Score: 0.3299
Document ID: 672c4855d2cd8acad1c18b56, Relevance Score: 0.3081
Document ID: 672c481ad2cd8acad1c08d88, Relevance Score: 0.2949
Document ID: 672c480fd2cd8acad1c05649, Relevance Score: 0.2820
Document ID: 672c4839d2cd8acad1c111d0, Relevance Score: 0.2549
